In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

from numpy import array
from numpy import asarray
from numpy import zeros
import pandas as pd
import csv
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding


Using TensorFlow backend.


In [0]:
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])

In [0]:
t = Tokenizer()
t.fit_on_texts(docs)

In [0]:
vocab_size = len(t.word_index)+1

In [0]:
encoded_docs = t.texts_to_sequences(docs)
encoded_docs

[[6, 2],
 [3, 1],
 [7, 4],
 [8, 1],
 [9],
 [10],
 [5, 4],
 [11, 3],
 [5, 1],
 [12, 13, 2, 14]]

In [0]:
max_len = 4
padded_docs = pad_sequences(encoded_docs,maxlen=max_len,padding='post')

In [0]:
padded_docs

array([[ 6,  2,  0,  0],
       [ 3,  1,  0,  0],
       [ 7,  4,  0,  0],
       [ 8,  1,  0,  0],
       [ 9,  0,  0,  0],
       [10,  0,  0,  0],
       [ 5,  4,  0,  0],
       [11,  3,  0,  0],
       [ 5,  1,  0,  0],
       [12, 13,  2, 14]], dtype=int32)

In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -uq glove*.zip -d "drive/My Drive/PreTrainedGloveVec"
!ls
!pwd
!rm glove.6B.200d.txt

--2019-03-26 09:52:12--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-03-26 09:52:13--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  6.28MB/s    in 3m 50s  

2019-03-26 09:56:03 (3.58 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [0]:
#File to Required Embedding Matrix
embedding_matrix_200_loaded = pd.read_table("drive/My Drive/PreTrainedGloveVec/glove.6B.200d.txt", sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
embedding_matrix = zeros((vocab_size, 200))
for word, i in t.word_index.items():
	embedding_vector = embedding_matrix_200_loaded.loc[word]
	if embedding_vector is not None:
		embedding_matrix[i] = asarray(embedding_vector)

In [0]:
model = Sequential()
e = Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 200)            3000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 801       
Total params: 3,801
Trainable params: 801
Non-trainable params: 3,000
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Accuracy: 100.000000


In [0]:
docs

['Well done!',
 'Good work',
 'Great effort',
 'nice work',
 'Excellent!',
 'Weak',
 'Poor effort!',
 'not good',
 'poor work',
 'Could have done better.']

In [0]:
from gensim.models import Word2Vec
#model = Word2Vec(docs, min_count=2,size=3,workers=2,sg=1)

In [0]:
# summarize vocabulary size in model
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 16


In [0]:
words

['W',
 'e',
 'l',
 ' ',
 'd',
 'o',
 'n',
 '!',
 'G',
 'w',
 'r',
 'k',
 'a',
 't',
 'f',
 'c']

In [0]:
model.train(docs, total_examples=len(docs), epochs=10)

(135, 1060)

In [0]:
words = list(model.wv.vocab)
print(words)

['W', 'e', 'l', ' ', 'd', 'o', 'n', '!', 'G', 'w', 'r', 'k', 'a', 't', 'f', 'c']


In [0]:
print(model['W'])

[ 0.07157493  0.01587445 -0.1180822 ]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [0]:
model.wv.save_word2vec_format('drive/My Drive/model.txt', binary=False)


In [0]:
f = open("model.txt")
f.read().split('\n')

['16 3',
 'o 0.16601144 0.08561262 -0.06584102',
 'e -0.085353225 -0.04919885 -0.025375495',
 '  0.13088718 -0.097947605 -0.16550545',
 'r -0.012047082 0.0024761453 -0.12620772',
 't -0.09419559 0.10935257 0.11428868',
 'l -0.014555561 0.039651327 -0.14230782',
 'd -0.012478611 -0.00930056 -0.09786297',
 'n -0.07102371 -0.03520095 0.074346095',
 'k 0.079257354 0.09521919 0.08769069',
 'f 0.007855951 -0.05572672 0.1226674',
 '! -0.0063268426 0.15560475 0.12192204',
 'w 0.024844185 -0.09503921 -0.0027861109',
 'a 0.13526775 -0.06588608 -0.15385665',
 'W 0.07157493 0.01587445 -0.1180822',
 'G 0.029843388 -0.11805015 -0.024976857',
 'c -0.07718378 -0.16658416 -0.08741212',
 '']

In [0]:
model[model.wv.vocab]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([[ 0.07157493,  0.01587445, -0.1180822 ],
       [-0.08535323, -0.04919885, -0.02537549],
       [-0.01455556,  0.03965133, -0.14230782],
       [ 0.13088718, -0.0979476 , -0.16550545],
       [-0.01247861, -0.00930056, -0.09786297],
       [ 0.16601144,  0.08561262, -0.06584102],
       [-0.07102371, -0.03520095,  0.0743461 ],
       [-0.00632684,  0.15560475,  0.12192204],
       [ 0.02984339, -0.11805015, -0.02497686],
       [ 0.02484418, -0.09503921, -0.00278611],
       [-0.01204708,  0.00247615, -0.12620772],
       [ 0.07925735,  0.09521919,  0.08769069],
       [ 0.13526775, -0.06588608, -0.15385665],
       [-0.09419559,  0.10935257,  0.11428868],
       [ 0.00785595, -0.05572672,  0.1226674 ],
       [-0.07718378, -0.16658416, -0.08741212]], dtype=float32)

In [0]:
# define training data
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
			['this', 'is', 'the', 'second', 'sentence'],
			['yet', 'another', 'sentence'],
			['one', 'more', 'sentence'],
			['and', 'the', 'final', 'sentence']]
# train model
model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
print(model['sentence'])
# save model
model.save('drive/My Drive/model.bin')
# load model
new_model = Word2Vec.load('drive/My Drive/model.bin')
print(new_model)

Word2Vec(vocab=14, size=100, alpha=0.025)
['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec', 'second', 'yet', 'another', 'one', 'more', 'and', 'final']
[ 2.0021577e-03  1.1767276e-03 -4.8685959e-04  4.0138788e-03
  2.0246159e-03 -4.5083119e-03 -1.7899695e-03  1.5989923e-03
 -2.1147723e-03 -2.0247502e-03 -2.6409226e-03 -4.9665896e-03
 -2.8930365e-03  1.5060665e-03  4.8331516e-03 -4.9222521e-03
 -4.9345689e-03 -3.8198475e-03  2.0167439e-03  3.9005089e-03
 -4.9091275e-03 -4.9269944e-03  4.5096376e-03  1.2305533e-03
 -2.9920300e-03  1.0258644e-03  1.2583132e-03 -1.6352446e-03
 -2.8725728e-06  1.2228074e-03  2.4809893e-03 -2.0063908e-03
 -4.7322214e-03 -3.9184294e-03  2.7595372e-03  3.5574725e-03
  4.6200687e-03  1.8202090e-03  7.6989172e-04  1.5749696e-03
 -4.4854204e-03 -4.4126301e-03  4.7863969e-03  1.2066915e-03
 -4.2836536e-03  4.5555751e-03  3.8166514e-03  1.7631321e-03
  2.8800410e-03 -2.2091288e-03  1.6481306e-03 -1.7723715e-03
  4.6610393e-04 -2.4996332e-03 -4.4883019e-0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
new_model.wv['and']

array([-0.00405814, -0.00099714, -0.00242968,  0.00063693, -0.00058246,
       -0.00173328, -0.00157168,  0.00288855,  0.00462457, -0.00450954,
        0.00060216,  0.00347092, -0.00022496,  0.00449333,  0.00211608,
       -0.0023389 ,  0.00429818, -0.0005854 ,  0.00394441, -0.00173315,
       -0.00089071, -0.00467051, -0.0006578 ,  0.00233846, -0.00373957,
       -0.00164598,  0.00016287,  0.00155444, -0.00371833, -0.00074708,
        0.00326871, -0.00439669,  0.00133605, -0.00163115, -0.00227549,
       -0.00449238, -0.00112025, -0.00294652, -0.00318599, -0.00218444,
        0.00056363,  0.00434647,  0.00247085,  0.00415618, -0.00077819,
       -0.00318875, -0.00310867,  0.00375308,  0.00351412, -0.00453365,
       -0.00359887,  0.00251142, -0.00429069, -0.00023212, -0.00333932,
        0.00213613, -0.00342835,  0.00335315,  0.00142852, -0.00145421,
       -0.00013683, -0.00352884,  0.00241563,  0.00286775, -0.00281557,
        0.00087539, -0.00215191,  0.00437363, -0.00069822,  0.00